**DB Timetable Scrapper**

Data Scrapping project to determine the punctuality of Deutsche Bahn at Münster main station. The timetable API from Deutsche Bahn is automatically scrapped in a two-minute rhythm to compare the planned timetable with the actual arrival and departure times.

In [1]:
import requests
import pandas as pd
import time
import xml.etree.ElementTree as et
import numpy as np
import datetime as dt
from datetime import datetime
from time import sleep

In [2]:
 plan = requests.get("https://api.deutschebahn.com/timetables/v1/plan/8011160/210604/10",
                    headers = {"Authorization": "Bearer #"})

In [3]:
plan

<Response [200]>

Train plans are available up to 19 hours into the future. All available plans are subsequently retrieved and merged. Queries are limited to 20 per minute.

In [6]:
#Filtering most important information and retrun dict consisting of xml responses.

def station_plan_dict(xml):

    station = []

    for i in range(len(xml)):
        train = {}
        train["id"] = xml[i].attrib["id"]
        train["train_type"] = xml[i][0].attrib["c"]
        try:
            train["arrival_plan"] = xml[i][1].attrib["pt"]
        except IndexError:
            train["arrival_plan"] = np.nan
        try:
            train["depature_plan"] = xml[i][2].attrib["pt"]
        except IndexError:
            train["depature_plan"] = np.nan
        station.append(train)
        
    return station

def padded_hour(h):
    if len(str(h)) < 2:
        return str("0{}".format(str(h)))
    else:
        return int(h)

def next_19_hours():
    time_slice = []
    hour = int(datetime.now().strftime("%H"))
    day = int(datetime.now().strftime("%d"))
    month = int(datetime.now().strftime("%m"))
    f = lambda x: "0{}".format(str(x)) if len(str(x)) < 2 else str(x)
    for i in range(19):
        if hour < 24:
            time_slice.append([f(day),f(hour),f(month)])
            hour += 1
        else:
            day += 1
            hour = 0
            time_slice.append([f(day),f(hour), f(month)])
            hour += 1
    return time_slice

def create_plan(station_eva=8000263):

    time_slice = next_19_hours()
    station_plan_dfs = []
    test_nr = 0
    global h_initialisation
    h_initialisation = int(datetime.now().strftime("%H"))
    d_initialisation = int(datetime.now().strftime("%d"))
    global upgrade_h
    global upgrade_day 
    if (int(h_initialisation) + 19) > 23:
        upgrade_h = int(h_initialisation) + 19 - 24
        upgrade_day = int((dt.datetime.now() + dt.timedelta(hours=24)).strftime("%d"))
    else:
        upgrade_h = int(h_initialisation) + 19
        upgrade_day = int(d_initialisation)
    
    for d,h,m in time_slice:
        url = "https://api.deutschebahn.com/timetables/v1/plan/{}/21{}{}/{}".format(station_eva,m,d,h)
        ret =  requests.get(url, headers = {"Authorization": "Bearer #"})
        xml = et.fromstring(ret.text)
        station_dict = station_plan_dict(xml)
        frame = pd.DataFrame(station_dict)
        station_plan_dfs.append(frame)
        print(ret.status_code,"for hour",h)
        test_nr += ret.status_code
    
    if test_nr == 200*19:
        print("---completed---")
    
    return station_plan_dfs

In [9]:
station_plan_dfs = create_plan()

200 for hour 09
200 for hour 10
200 for hour 11
200 for hour 12
200 for hour 13
200 for hour 14
200 for hour 15
200 for hour 16
200 for hour 17
200 for hour 18
200 for hour 19
200 for hour 20
200 for hour 21
200 for hour 22
200 for hour 23
200 for hour 00
200 for hour 01
200 for hour 02
200 for hour 03
---completed---


In [10]:
planned = pd.concat(station_plan_dfs,ignore_index=True)
planned = planned.drop_duplicates(subset="id")
planned

,id,train_type,arrival_plan,depature_plan
0,-9156762936336497031-2106040814-11,RB,2106040945,NaN
1,-3239106104198922327-2106040843-6,Bus,2106040959,NaN
2,6065987942969979065-2106040903-1,RB,2106040903,NaN
3,6467741671241730152-2106040628-7,EC,2106040900,2106040903
4,6072791769348791595-2106040919-8,RB,2106040955,NaN
...,...,...,...,...
512,7682710902566184985-2106050106-18,RE,2106050253,NaN
513,-2371128027070490466-2106050235-1,RB,2106050235,NaN
514,-6058665356135913865-2106050209-1,RE,2106050209,NaN
515,2748297611992163640-2106050135-11,RB,2106050222,NaN


Deviations from the train schedule are provided only for future stops. When a train arrives at the station 
arrives, the delay for the reached stop is no longer listed. There are two 
ports for delays:

* fchg lists all known future delays.
* rchg lists all delays that have been reported in the last two minutes.
   
In a first step, fchg is used to initalize the schedule.
rchg is used to update the schedule every two minutes

In [17]:
xml_all_changes = requests.get("https://api.deutschebahn.com/timetables/v1/fchg/8000263",
                   headers = {"Authorization": "Bearer #"}) 

In [18]:
xml_all_changes

<Response [200]>

In [19]:
xml_change  = et.fromstring(xml_all_changes.text)

In [20]:
def changes_dict(xml):
    all_station_changes = []
    for i in range(len(xml)):
        change = {}
        change["id"] = xml[i].attrib["id"]
        try:
            change["arrival_change"] =  xml[i].findall(".//ar")[0].attrib["ct"]
        except:
            change["arrival_change"] = np.nan
        try:
            change["depature_change"] =  xml[i].findall(".//dp")[0].attrib["ct"]
        except:
            change["depature_change"] = np.nan
        
        all_station_changes.append(change)
    
    return all_station_changes

def create_change_df(dic):
    frame = pd.DataFrame(dic)
    ar_null = frame["arrival_change"].apply(pd.isnull)
    dp_null = frame["depature_change"].apply(pd.isnull)
    both_null = []
    for i in range(len(ar_null)):        
        if ar_null[i] == True & dp_null[i] == True:
            both_null.append(False)
        else:
            both_null.append(True)
    frame = frame.loc[both_null,:].reset_index(drop=True)

    return frame

In [21]:
all_changes = changes_dict(xml_change)
all_changes = create_change_df(all_changes)

In [22]:
data = planned.merge(all_changes,how="left",on="id")
data

,id,train_type,arrival_plan,depature_plan,arrival_change,depature_change
0,-9156762936336497031-2106040814-11,RB,2106040945,NaN,2106040948,NaN
1,-3239106104198922327-2106040843-6,Bus,2106040959,NaN,NaN,NaN
2,6065987942969979065-2106040903-1,RB,2106040903,NaN,NaN,2106040906
3,6467741671241730152-2106040628-7,EC,2106040900,2106040903,2106040904,2106040906
4,6072791769348791595-2106040919-8,RB,2106040955,NaN,2106040955,NaN
...,...,...,...,...,...,...
508,7682710902566184985-2106050106-18,RE,2106050253,NaN,NaN,NaN
509,-2371128027070490466-2106050235-1,RB,2106050235,NaN,NaN,NaN
510,-6058665356135913865-2106050209-1,RE,2106050209,NaN,NaN,NaN
511,2748297611992163640-2106050135-11,RB,2106050222,NaN,NaN,NaN


In [23]:
def set_depature(ar_p,dp_p):
    i = 0
    update = []
    boola = dp_p.isnull()
    for bo in boola:
        if bo == True:
            update.append(ar_p[i])
        else:
            update.append(dp_p[i])
        i += 1
    return update


def detect_equal_times(p,c):
    i=0
    update = []
    for i in range(len(p)):
        if p[i] == c[i]:
            update.append("No")
        else:
            update.append(c[i])
    return update

def ar_diff(plan):
    i=0
    diff = []
    for i in range(len(plan)):
        try:
            sub = data["arrival_change"][i] - plan[i]
            diff.append(sub)
            i += 1
        except:
            diff.append(0)
            i += 1
            
    return diff

In [24]:
#If there is no departure time, then it is the last stop. Departure is set equal to arrival
data["depature_plan"] = set_depature(data["arrival_plan"],data["depature_plan"])
#If the arrival time is equal to the changed arrival time, then other properties of the trip have changed
#(e.g.: platform). Change is reset to "No".
data["arrival_change"] = detect_equal_times(data["arrival_plan"],data["arrival_change"])
data["depature_change"] = detect_equal_times(data["depature_plan"],data["depature_change"])
# If Change is NAN its set to "NO"
data["arrival_change"] = data["arrival_change"].apply(lambda x: x if type(x) == str else "No")
data["depature_change"] = data["depature_change"].apply(lambda x: x if type(x) == str else "No")

In [25]:
# transforming str to datetime object
data["arrival_plan"] = data["arrival_plan"].apply(pd.to_datetime,errors="ignore",format="%y%m%d%H%M")
data["arrival_change"] = data["arrival_change"].apply(pd.to_datetime,errors="ignore",format="%y%m%d%H%M")
data["depature_plan"] = data["depature_plan"].apply(pd.to_datetime,errors="ignore",format="%y%m%d%H%M")
data["depature_change"] = data["depature_change"].apply(pd.to_datetime,errors="ignore",format="%y%m%d%H%M")

In [26]:
#Calculate Delay
data["min_delayed"] = ar_diff(data["arrival_plan"])
data["min_delayed"] = data["min_delayed"].apply(lambda x: 0 if type(x)==int else x.total_seconds()/60)
#DB defienes trains as delayed if the train is more than six minutes delayed 
data["more_than_6"] = data["min_delayed"].apply(lambda x: 0 if x < 6 else 1)
data["is_delayed"] = data["min_delayed"].apply(lambda x: 1 if x > 0 else 0)

In [27]:
data.sort_values("arrival_plan", inplace = True, ignore_index= True)
data 

,id,train_type,arrival_plan,depature_plan,arrival_change,depature_change,min_delayed,more_than_6,is_delayed
0,4099684208263039472-2106040605-10,IC,2021-06-04 08:58:00,2021-06-04 09:00:00,2021-06-04 09:01:00,2021-06-04 09:04:00,3.0,0,1
1,6467741671241730152-2106040628-7,EC,2021-06-04 09:00:00,2021-06-04 09:03:00,2021-06-04 09:04:00,2021-06-04 09:06:00,4.0,0,1
2,6065987942969979065-2106040903-1,RB,2021-06-04 09:03:00,2021-06-04 09:03:00,No,2021-06-04 09:06:00,0.0,0,0
3,1008060119682466127-2106040904-1,Bus,2021-06-04 09:04:00,2021-06-04 09:04:00,No,No,0.0,0,0
4,-4590997021422529160-2106040902-2,RB,2021-06-04 09:05:00,2021-06-04 09:08:00,2021-06-04 09:06:00,No,1.0,0,1
...,...,...,...,...,...,...,...,...,...
508,-6058665356135913865-2106050209-1,RE,2021-06-05 02:09:00,2021-06-05 02:09:00,No,No,0.0,0,0
509,2748297611992163640-2106050135-11,RB,2021-06-05 02:22:00,2021-06-05 02:22:00,No,No,0.0,0,0
510,-2371128027070490466-2106050235-1,RB,2021-06-05 02:35:00,2021-06-05 02:35:00,No,No,0.0,0,0
511,7682710902566184985-2106050106-18,RE,2021-06-05 02:53:00,2021-06-05 02:53:00,No,No,0.0,0,0


In [33]:
#the timetable is updated every two minutes
#new train schedules are appendet to the timetabele as soon as they are published

def update_changes(eva=8000263):
    
    check_delayed = lambda x: 1 if x > 0 else 0
    check_6_delayed = lambda x: 1 if x >= 6 else 0
    
    global upgrade_h
    global upgrade_day
    global h_initialisation
    
    month = next_19_hours()[-1][-1]
    if (int(datetime.now().strftime("%H")) == (h_initialisation + 1)) & ((int(datetime.now().strftime("%M")) == 3) | (int(datetime.now().strftime("%M")) == 2)):
        
        
        print("data expanded for hour {}".format(upgrade_h))
        url = "https://api.deutschebahn.com/timetables/v1/plan/{}/21{}{}/{}".format(
            eva,month,upgrade_day,padded_hour(upgrade_h))
        ret =  requests.get(url, headers = {"Authorization": "Bearer #"})
        xml = et.fromstring(ret.text) #new
        station_dict = station_plan_dict(xml)
        frame = pd.DataFrame(station_dict)
        frame["arrival_change"] = "No"
        frame["depature_change"] = "No"
        frame["min_delayed"] = 0
        frame["more_than_6"] = 0
        frame["is_delayed"] = 0
        frame["depature_plan"] = set_depature(frame["arrival_plan"],frame["depature_plan"])
        frame["depature_plan"] = frame["depature_plan"].apply(pd.to_datetime,errors="ignore",format="%y%m%d%H%M")
        frame["arrival_plan"] = frame["arrival_plan"].apply(pd.to_datetime,errors="ignore",format="%y%m%d%H%M")
        global data
        frame.sort_values("arrival_plan", inplace = True, ignore_index= True)
        data = pd.concat([data,frame],ignore_index=True)
        data.drop_duplicates(subset = "id", inplace = True,ignore_index = True)
        
        if (upgrade_h + 1) > 23:
            upgrade_h = 0
            upgrade_day = int((dt.datetime.now() + dt.timedelta(hours=24)).strftime("%d"))
            h_initialisation = int((datetime.now()).strftime("%H"))
            
        else:
            upgrade_h += 1
            h_initialisation = int((datetime.now()).strftime("%H"))
    
    
    url = "https://api.deutschebahn.com/timetables/v1/rchg/{}".format(eva)
    ret = requests.get(url, headers = {"Authorization": "Bearer #"})
    xml_change  = et.fromstring(ret.text)

    changes = changes_dict(xml_change)
    changes = create_change_df(changes)
    
    changes["arrival_change"] = changes["arrival_change"].apply(lambda x: x if type(x) == str else "No")
    changes["depature_change"] = changes["depature_change"].apply(lambda x: x if type(x) == str else "No")

    changes["arrival_change"] = changes["arrival_change"].apply(pd.to_datetime,errors="ignore",format="%y%m%d%H%M")
    changes["depature_change"] = changes["depature_change"].apply(pd.to_datetime,errors="ignore",format="%y%m%d%H%M")
    

    for change in changes.iterrows():
        
        try:
            if change[1]["arrival_change"] != "No":
                data.loc[data["id"] == change[1]["id"],"arrival_change"] = change[1]["arrival_change"]
                delay = change[1]["arrival_change"] - data.loc[data["id"]== change[1]["id"],"arrival_plan"]
                data.loc[data["id"]== change[1]["id"],"min_delayed"] = delay.iloc[0].total_seconds()/60
                data.loc[data["id"]== change[1]["id"],"is_delayed"] = check_delayed(data.loc[data["id"]== change[1]["id"],"min_delayed"].iloc[0])
                data.loc[data["id"]== change[1]["id"],"more_than_6"] = check_6_delayed(data.loc[data["id"]== change[1]["id"],"min_delayed"].iloc[0])
                
            if change[1]["depature_change"] != "No":
                data.loc[data["id"] == change[1]["id"],"depature_change"] = change[1]["depature_change"]
                
        except:
            
            pass
            
    return None

In [34]:
update_changes()

In [35]:
data

,id,train_type,arrival_plan,depature_plan,arrival_change,depature_change,min_delayed,more_than_6,is_delayed
0,4099684208263039472-2106040605-10,IC,2021-06-04 08:58:00,2021-06-04 09:00:00,2021-06-04 09:01:00,2021-06-04 09:04:00,3.0,0,1
1,6467741671241730152-2106040628-7,EC,2021-06-04 09:00:00,2021-06-04 09:03:00,2021-06-04 09:04:00,2021-06-04 09:06:00,4.0,0,1
2,6065987942969979065-2106040903-1,RB,2021-06-04 09:03:00,2021-06-04 09:03:00,No,2021-06-04 09:06:00,0.0,0,0
3,1008060119682466127-2106040904-1,Bus,2021-06-04 09:04:00,2021-06-04 09:04:00,No,No,0.0,0,0
4,-4590997021422529160-2106040902-2,RB,2021-06-04 09:05:00,2021-06-04 09:08:00,2021-06-04 09:06:00,No,1.0,0,1
...,...,...,...,...,...,...,...,...,...
508,-6058665356135913865-2106050209-1,RE,2021-06-05 02:09:00,2021-06-05 02:09:00,No,No,0.0,0,0
509,2748297611992163640-2106050135-11,RB,2021-06-05 02:22:00,2021-06-05 02:22:00,No,No,0.0,0,0
510,-2371128027070490466-2106050235-1,RB,2021-06-05 02:35:00,2021-06-05 02:35:00,No,No,0.0,0,0
511,7682710902566184985-2106050106-18,RE,2021-06-05 02:53:00,2021-06-05 02:53:00,No,No,0.0,0,0


In [ ]:
while True:
    
    try: 
        
        start_time = time.time()
        end_time = start_time + 120
        update_changes()
        if int(datetime.now().strftime("%M")) == 31:
            data.to_csv("date_{}.csv".format(datetime.now().strftime("%H_%M")))
            print("-------Saved------------")
        sleep(end_time - time.time())
        
    except:
        
        break 

Tabel is saved and keept for later analysis.